### Import Dependencies

In [29]:
import os
import detectron2
import cv2
import random
import matplotlib.pyplot as plt
import pickle
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()
import math

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data import build_detection_test_loader, build_detection_train_loader

### Register Dataset

In [3]:
# format: register_coco_instances(name, metadata, json_file, image_root):
# name = dataset name
# once you have registered the dataset, you can use the name of the dataset in cfg.DATASETS.{TRAIN,TEST}
register_coco_instances(name='coco_train1', metadata={}, json_file='../../dataset/train.json', image_root='../../dataset/')
register_coco_instances(name='coco_test1',  metadata={}, json_file='../../dataset/test.json',  image_root='../../dataset/')

# revise metadata
MetadataCatalog.get('coco_train1').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

### Load and Revise Config File

In [4]:
# load config file
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))

# revise config file
cfg.DATASETS.TRAIN = ('coco_train1', {}, '../../dataset/train.json', '../../dataset/')
cfg.DATASETS.TEST  = ('coco_test1', {}, '../../dataset/test.json', '../../dataset/')

### Create a batch of 16 images (Only Used For group_image Helper Method) and Generate Pickle File

In [93]:
# Takes 10 minutes to complete
# If you're hesitant to wait, then just load pickle the file I provided to you. No need to run this cell!
import pickle

my_dataset_train_metadata = MetadataCatalog.get("coco_train1")
dataset_dicts = DatasetCatalog.get("coco_train1")

images_dataset  = []
image_collector = []
for idx, data in enumerate(dataset_dicts): # takes 10 minutes
    img_data = cv2.imread(data["file_name"]) 
    visualizer = Visualizer(img_data[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.3)
    vis = visualizer.draw_dataset_dict(data)
    image = vis.get_image()[:, :, ::-1]
    image_collector.append(image)
    
    if idx % 16 == 0:
        images_dataset.append(image_collector)
        image_collector = []

with open('image_dataset.txt', 'wb') as f: # create the file in the current directory
    pickle.dump(images_dataset, f)

[05/04 16:27:42 d2.data.datasets.coco]: Loaded 4883 images in COCO format from ../../dataset/train.json


### Load Pickle File

In [5]:
with open('image_dataset.txt', 'rb') as f:
    loaded_pickle = pickle.load(f)

# if the output is 306, your good to go!
print(len(loaded_pickle))

306


In [13]:
# output of dataset_dicts
# {'file_name': '../../dataset/train/4183.jpg', 'height': 1024, 'width': 1024, 'image_id': 4183, 'annotations': [{'iscrowd': 0, 'bbox': [63.3, 235.9, 706.0, 559.0], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}]}

### 3 Helper Methods!!

In [63]:
# group image
def group_image(images_dataset, idx):
    '''
    idx 0: 0000
    idx 1: 0001~0016
    idx 2: 0017~0032
    '''
    fig, axes = plt.subplots(4, 4, figsize=(12,12), dpi=120)
    axes = axes.flatten()

    num = 0
    if idx == 0:
        picture_number = [0]
    else:
        picture_number = [i for i in range((idx*16)-15, ((idx+1)*16)-15)]
    for idx, (i, pic_num) in enumerate(zip(images_dataset[idx], picture_number), start=num):
        pic_num = str(pic_num).zfill(4)
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        axes[idx].imshow(i)
        axes[idx].set_title(f"train/{pic_num}.jpg")
        axes[idx].axis('off')

# individual image
def individual_image(train_dataset, id:int): 
    original_number = id 
    id = math.ceil(id/16)
    picture_number = [i for i in range((id*16)-15, ((id+1)*16)-15)]
    train_dataset = train_dataset[id]
    for idx, data in enumerate(train_dataset):
        if idx == original_number - picture_number[0]:
            data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
            plt.figure()
            plt.title(f"train/{str(original_number).zfill(4)}.jpg")
            plt.imshow(data)
            plt.axis('off')

# random image
def random_image(number_of_images:int=5):
    my_dataset_train_metadata = MetadataCatalog.get("coco_train1")
    dataset_dicts = DatasetCatalog.get("coco_train1")
    for data in random.sample(dataset_dicts, number_of_images):
        img_data = cv2.imread(data["file_name"]) 
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
        visualizer = Visualizer(img_data[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(data)
        image = vis.get_image()[:, :, ::-1]
        plt.figure()
        plt.imshow(image)
        plt.title(f"{data['file_name'][-14:]}")
        plt.axis('off')

### DEMO

In [1]:
# random images (it will show N images)
random_image(3)

NameError: name 'random_image' is not defined

In [2]:
# individual image (select from 0000~4882)
individual_image(loaded_pickle, 1234)

NameError: name 'individual_image' is not defined

In [3]:
# group image (select from 0~305)
# 0 = 0000 (only)
# 1 = 0001 ~ 0016 
# 2 = 0017 ~ 0032 
# and so on..
group_image(loaded_pickle, 78)

NameError: name 'group_image' is not defined